[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ClimateMatchAcademy/course-content/blob/main/tutorials/W1D1_ClimateSystemOverview/student/W1D1_Tutorial1.ipynb) &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/tutorials/W1D1_ClimateSystemOverview/student/W1D1_Tutorial1.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>


# Tutorial 1: Creating DataArrays and Datasets to Assess Global Climate Data

**Week 1, Day 1, Climate System Overview**

**Content creators:** Sloane Garelick, Julia Kent

**Content reviewers:** Yosmely Bermúdez, Katrina Dobson, Younkap Nina Duplex, Danika Gupta, Maria Gonzalez, Will Gregory, Nahid Hasan, Sherry Mi, Beatriz Cosenza Muralles, Jenna Pearson, Chi Zhang, Ohad Zivan

**Content editors:** Jenna Pearson, Chi Zhang, Ohad Zivan

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS and Google DeepMind


## ![project pythia](https://projectpythia.org/_static/images/logos/pythia_logo-blue-rtext.svg)

Pythia credit: Rose, B. E. J., Kent, J., Tyle, K., Clyne, J., Banihirwe, A., Camron, D., May, R., Grover, M., Ford, R. R., Paul, K., Morley, J., Eroglu, O., Kailyn, L., & Zacharias, A. (2023). Pythia Foundations (Version v2023.05.01) https://zenodo.org/record/8065851

## ![CMIP.png](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/Art/CMIP.png?raw=true)


# Tutorial Objectives

As you just learned in the Introduction to Climate video, variations in global climate involve various forcings, feedbacks, and interactions between multiple processes and systems. Because of this complexity, global climate datasets are often very large with multiple dimensions and variables.

One useful computational tool for organizing, analyzing and interpreting large global datasets is [Xarray](https://xarray.pydata.org/en/v2023.05.0/getting-started-guide/why-xarray.html), an open source project and Python package that makes working with labelled multi-dimensional arrays simple and efficient.

In this first tutorial, we will use the `DataArray` and `Dataset` objects, which are used to represent and manipulate spatial data, to practice organizing large global climate datasets and to understand variations in Earth's climate system.


# Setup


Similar to `numpy`, `np`; `pandas`, `pd`; you may often encounter `xarray` imported within a shortened namespace as `xr`.


In [1]:
# imports
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt


In [2]:
# @title Figure Settings
import ipywidgets as widgets  # interactive display

%config InlineBackend.figure_format = 'retina'
plt.style.use(
    "https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle"
)


In [3]:
# @title Video 1: Introduction to Climate

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
    def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
        self.id = id
        if source == "Bilibili":
            src = f"https://player.bilibili.com/player.html?bvid={id}&page={page}"
        elif source == "Osf":
            src = f"https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render"
        super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
    tab_contents = []
    for i, video_id in enumerate(video_ids):
        out = widgets.Output()
        with out:
            if video_ids[i][0] == "Youtube":
                video = YouTubeVideo(
                    id=video_ids[i][1], width=W, height=H, fs=fs, rel=0
                )
                print(f"Video available at https://youtube.com/watch?v={video.id}")
            else:
                video = PlayVideo(
                    id=video_ids[i][1],
                    source=video_ids[i][0],
                    width=W,
                    height=H,
                    fs=fs,
                    autoplay=False,
                )
                if video_ids[i][0] == "Bilibili":
                    print(
                        f"Video available at https://www.bilibili.com/video/{video.id}"
                    )
                elif video_ids[i][0] == "Osf":
                    print(f"Video available at https://osf.io/{video.id}")
            display(video)
        tab_contents.append(out)
    return tab_contents


video_ids = [("Youtube", "mc-DkvYLdOA"), ("Bilibili", "BV1Th4y1j7SS")]
tab_contents = display_videos(video_ids, W=730, H=410)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
    tabs.set_title(i, video_ids[i][0])
display(tabs)

In [4]:
# @title Tutorial slides
# @markdown These are the slides for the videos in all tutorials today
from IPython.display import IFrame

link_id = "4suf5"

# Introducing the `DataArray` and `Dataset`

[Xarray](https://xarray.pydata.org/en/v2023.05.0/getting-started-guide/why-xarray.html) expands on the capabilities on [NumPy](https://numpy.org/doc/stable/user/index.html#user) arrays, providing a lot of streamlined data manipulation. It is similar in that respect to [Pandas](https://pandas.pydata.org/docs/user_guide/index.html#user-guide), but whereas Pandas excels at working with tabular data, Xarray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's [Common Data Model (CDM)](https://docs.unidata.ucar.edu/netcdf-java/current/userguide/common_data_model_overview.html).


# Section 1: Creation of a `DataArray` Object

The `DataArray` is one of the basic building blocks of Xarray (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray)). It provides a `numpy.ndarray`-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

Here we'll initialize a `DataArray` object by wrapping a plain NumPy array, and explore a few of its properties.


## Section 1.1: Generate a Random Numpy Array

For our first example, we'll just create a random array of "temperature" data in units of Kelvin:


In [5]:
rand_data = 283 + 5 * np.random.randn(5, 3, 4)
rand_data


array([[[269.43608798, 280.16719877, 283.27360116, 282.12651778],
        [283.10080511, 277.59885105, 284.09822432, 284.05196465],
        [278.63903025, 277.56385454, 277.81928646, 289.68116922]],

       [[283.00057474, 287.71001837, 286.05930537, 282.37957512],
        [290.16326565, 283.18119117, 283.89720995, 281.90510543],
        [280.84638052, 283.84034614, 284.49467247, 282.45617761]],

       [[289.92876497, 287.74142637, 286.42577665, 280.01475134],
        [277.99606489, 285.90054595, 284.27371294, 290.18164011],
        [285.55792131, 289.96820992, 282.7313579 , 291.58823766]],

       [[283.91345209, 284.350952  , 284.18870832, 288.61411588],
        [289.72093571, 282.53313909, 279.46477172, 286.01378953],
        [283.75845529, 279.90114591, 286.09904789, 290.46263724]],

       [[279.00446428, 282.65122128, 280.83979128, 286.16955388],
        [286.3668465 , 283.82645456, 283.76878833, 281.57682055],
        [284.3748313 , 287.94328877, 285.61567639, 280.73052086]]])

## Section 1.2: Wrap the Array: First Attempt

Now we create a basic `DataArray` just by passing our plain `data` as an input:


In [6]:
temperature = xr.DataArray(rand_data)
temperature


<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[269.43608798, 280.16719877, 283.27360116, 282.12651778],
        [283.10080511, 277.59885105, 284.09822432, 284.05196465],
        [278.63903025, 277.56385454, 277.81928646, 289.68116922]],

       [[283.00057474, 287.71001837, 286.05930537, 282.37957512],
        [290.16326565, 283.18119117, 283.89720995, 281.90510543],
        [280.84638052, 283.84034614, 284.49467247, 282.45617761]],

       [[289.92876497, 287.74142637, 286.42577665, 280.01475134],
        [277.99606489, 285.90054595, 284.27371294, 290.18164011],
        [285.55792131, 289.96820992, 282.7313579 , 291.58823766]],

       [[283.91345209, 284.350952  , 284.18870832, 288.61411588],
        [289.72093571, 282.53313909, 279.46477172, 286.01378953],
        [283.75845529, 279.90114591, 286.09904789, 290.46263724]],

       [[279.00446428, 282.65122128, 280.83979128, 286.16955388],
        [286.3668465 , 283.82645456, 283.76878833, 281.57682055],
        [284.3748313 , 287.94328877, 285.61567639, 280.73052086]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

Note two things:

1. Xarray generates some basic dimension names for us (`dim_0`, `dim_1`, `dim_2`). We'll improve this with better names in the next example.
2. Wrapping the numpy array in a `DataArray` gives us a rich display in the notebook! (Try clicking the array symbol to expand or collapse the view)


## Section 1.3: Assign Dimension Names

Much of the power of Xarray comes from making use of named dimensions. So let's add some more useful names! We can do that by passing an ordered list of names using the keyword argument `dims`:


In [7]:
temperature = xr.DataArray(rand_data, dims=["time", "lat", "lon"])
temperature


<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[269.43608798, 280.16719877, 283.27360116, 282.12651778],
        [283.10080511, 277.59885105, 284.09822432, 284.05196465],
        [278.63903025, 277.56385454, 277.81928646, 289.68116922]],

       [[283.00057474, 287.71001837, 286.05930537, 282.37957512],
        [290.16326565, 283.18119117, 283.89720995, 281.90510543],
        [280.84638052, 283.84034614, 284.49467247, 282.45617761]],

       [[289.92876497, 287.74142637, 286.42577665, 280.01475134],
        [277.99606489, 285.90054595, 284.27371294, 290.18164011],
        [285.55792131, 289.96820992, 282.7313579 , 291.58823766]],

       [[283.91345209, 284.350952  , 284.18870832, 288.61411588],
        [289.72093571, 282.53313909, 279.46477172, 286.01378953],
        [283.75845529, 279.90114591, 286.09904789, 290.46263724]],

       [[279.00446428, 282.65122128, 280.83979128, 286.16955388],
        [286.3668465 , 283.82645456, 283.76878833, 281.57682055],
        [284.3748313 , 287.94328877, 285.61567639, 280.73052086]]])
Dimensions without coordinates: time, lat, lon

This is already an improvement over a NumPy array because we have names for each of the dimensions (or axes). Even better, we can associate arrays representing the values for the coordinates for each of these dimensions with the data when we create the `DataArray`. We'll see this in the next example.


# Section 2: Create a `DataArray` with Named Coordinates

## Section 2.1: Make Time and Space Coordinates

Here we will use [Pandas](https://foundations.projectpythia.org/core/pandas.html) to create an array of [datetime data](https://foundations.projectpythia.org/core/datetime.html), which we will then use to create a `DataArray` with a named coordinate `time`.


In [8]:
times_index = pd.date_range("2018-01-01", periods=5)
times_index


DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

We'll also create arrays to represent sample longitude and latitude:


In [9]:
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)


### Section 2.1.1: Initialize the `DataArray` with Complete Coordinate Info

When we create the `DataArray` instance, we pass in the arrays we just created:


In [10]:
temperature = xr.DataArray(
    rand_data, coords=[times_index, lats, lons], dims=["time", "lat", "lon"]
)
temperature


<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[269.43608798, 280.16719877, 283.27360116, 282.12651778],
        [283.10080511, 277.59885105, 284.09822432, 284.05196465],
        [278.63903025, 277.56385454, 277.81928646, 289.68116922]],

       [[283.00057474, 287.71001837, 286.05930537, 282.37957512],
        [290.16326565, 283.18119117, 283.89720995, 281.90510543],
        [280.84638052, 283.84034614, 284.49467247, 282.45617761]],

       [[289.92876497, 287.74142637, 286.42577665, 280.01475134],
        [277.99606489, 285.90054595, 284.27371294, 290.18164011],
        [285.55792131, 289.96820992, 282.7313579 , 291.58823766]],

       [[283.91345209, 284.350952  , 284.18870832, 288.61411588],
        [289.72093571, 282.53313909, 279.46477172, 286.01378953],
        [283.75845529, 279.90114591, 286.09904789, 290.46263724]],

       [[279.00446428, 282.65122128, 280.83979128, 286.16955388],
        [286.3668465 , 283.82645456, 283.76878833, 281.57682055],
        [284.3748313 , 287.94328877, 285.61567639, 280.73052086]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

### Section 2.1.2: Set Useful Attributes

We can also set some attribute metadata, which will help provide clear descriptions of the data. In this case, we can specify that we're looking at 'air_temperature' data and the units are 'kelvin'.


In [11]:
temperature.attrs["units"] = "kelvin"
temperature.attrs["standard_name"] = "air_temperature"

temperature


<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[269.43608798, 280.16719877, 283.27360116, 282.12651778],
        [283.10080511, 277.59885105, 284.09822432, 284.05196465],
        [278.63903025, 277.56385454, 277.81928646, 289.68116922]],

       [[283.00057474, 287.71001837, 286.05930537, 282.37957512],
        [290.16326565, 283.18119117, 283.89720995, 281.90510543],
        [280.84638052, 283.84034614, 284.49467247, 282.45617761]],

       [[289.92876497, 287.74142637, 286.42577665, 280.01475134],
        [277.99606489, 285.90054595, 284.27371294, 290.18164011],
        [285.55792131, 289.96820992, 282.7313579 , 291.58823766]],

       [[283.91345209, 284.350952  , 284.18870832, 288.61411588],
        [289.72093571, 282.53313909, 279.46477172, 286.01378953],
        [283.75845529, 279.90114591, 286.09904789, 290.46263724]],

       [[279.00446428, 282.65122128, 280.83979128, 286.16955388],
        [286.3668465 , 283.82645456, 283.76878833, 281.57682055],
        [284.3748313 , 287.94328877, 285.61567639, 280.73052086]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

### Section 2.1.3: Attributes Are Not Preserved by Default!

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

To illustrate this, we'll do a simple unit conversion from Kelvin to Celsius:


In [12]:
temperature_in_celsius = temperature - 273.15
temperature_in_celsius


<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[-3.71391202,  7.01719877, 10.12360116,  8.97651778],
        [ 9.95080511,  4.44885105, 10.94822432, 10.90196465],
        [ 5.48903025,  4.41385454,  4.66928646, 16.53116922]],

       [[ 9.85057474, 14.56001837, 12.90930537,  9.22957512],
        [17.01326565, 10.03119117, 10.74720995,  8.75510543],
        [ 7.69638052, 10.69034614, 11.34467247,  9.30617761]],

       [[16.77876497, 14.59142637, 13.27577665,  6.86475134],
        [ 4.84606489, 12.75054595, 11.12371294, 17.03164011],
        [12.40792131, 16.81820992,  9.5813579 , 18.43823766]],

       [[10.76345209, 11.200952  , 11.03870832, 15.46411588],
        [16.57093571,  9.38313909,  6.31477172, 12.86378953],
        [10.60845529,  6.75114591, 12.94904789, 17.31263724]],

       [[ 5.85446428,  9.50122128,  7.68979128, 13.01955388],
        [13.2168465 , 10.67645456, 10.61878833,  8.42682055],
        [11.2248313 , 14.79328877, 12.46567639,  7.58052086]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

We usually wish to keep metadata with our dataset, even after manipulating the data. For example it can tell us what the units are of a variable of interest. So when you perform operations on your data, make sure to check that all the information you want is carried over. If it isn't, you can add it back in following the instructions in the section before this. For an in-depth discussion of how Xarray handles metadata, you can find more information in the Xarray documents [here](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#approach-to-metadata).


# Section 3: The `Dataset`: a Container for `DataArray`s with Shared Coordinates

Along with `DataArray`, the other key object type in Xarray is the `Dataset`, which is a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset)).

The most common way to create a `Dataset` object is to load data from a file (which we will practice in a later tutorial). Here, instead, we will create another `DataArray` and combine it with our `temperature` data.

This will illustrate how the information about common coordinate axes is used.


## Section 3.1: Create a Pressure `DataArray` Using the Same Coordinates

For our next `DataArry` example, we'll create a random array of `pressure` data in units of hectopascal (hPa). This code mirrors how we created the `temperature` object above.


In [13]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times_index, lats, lons], dims=["time", "lat", "lon"]
)
pressure.attrs["units"] = "hPa"
pressure.attrs["standard_name"] = "air_pressure"

pressure

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[ 999.74093538,  998.31870626,  991.95144368, 1003.27487984],
        [ 999.34766328, 1000.99127862, 1004.41242884, 1001.31141224],
        [1003.71698511,  994.01681277, 1010.9312329 ,  998.11440997]],

       [[ 997.02633328,  999.53683948, 1002.09920067, 1003.90272932],
        [ 999.54427246,  994.26087343,  995.18843132, 1006.53230269],
        [1004.83661616,  995.95255326,  994.60954819,  995.94156946]],

       [[ 992.48242962,  997.10037573,  991.17681274,  992.20592869],
        [ 994.01125247,  988.22293807,  989.68967838,  993.393716  ],
        [ 990.66950931,  995.34289169,  999.16309764,  997.61933578]],

       [[1001.92128515,  992.21590895, 1001.03328445, 1002.37262482],
        [1002.1061563 ,  994.19933894,  997.34679402, 1002.51909023],
        [1002.14300631, 1003.47905083,  997.60738123, 1009.92212443]],

       [[ 992.28932444, 1003.51508065,  988.90427335, 1000.35929282],
        [1001.80190108,  996.03234018,  996.63591226, 1004.63443074],
        [ 993.13155643,  990.40057918,  991.26890895, 1005.32344238]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

## Section 3.2: Create a `Dataset` Object

Each `DataArray` in our `Dataset` needs a name!

The most straightforward way to create a `Dataset` with our `temperature` and `pressure` arrays is to pass a dictionary using the keyword argument `data_vars`:


In [14]:
ds = xr.Dataset(data_vars={"Temperature": temperature, "Pressure": pressure})
ds


<xarray.Dataset>
Dimensions:      (time: 5, lat: 3, lon: 4)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 25.0 40.0 55.0
  * lon          (lon) float64 -120.0 -100.0 -80.0 -60.0
Data variables:
    Temperature  (time, lat, lon) float64 269.4 280.2 283.3 ... 285.6 280.7
    Pressure     (time, lat, lon) float64 999.7 998.3 992.0 ... 991.3 1.005e+03

Notice that the `Dataset` object `ds` is aware that both data arrays sit on the same coordinate axes.


## Section 3.3: Access Data Variables and Coordinates in a `Dataset`

We can pull out any of the individual `DataArray` objects in a few different ways.

Using the "dot" notation:


In [15]:
ds.Pressure


<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[ 999.74093538,  998.31870626,  991.95144368, 1003.27487984],
        [ 999.34766328, 1000.99127862, 1004.41242884, 1001.31141224],
        [1003.71698511,  994.01681277, 1010.9312329 ,  998.11440997]],

       [[ 997.02633328,  999.53683948, 1002.09920067, 1003.90272932],
        [ 999.54427246,  994.26087343,  995.18843132, 1006.53230269],
        [1004.83661616,  995.95255326,  994.60954819,  995.94156946]],

       [[ 992.48242962,  997.10037573,  991.17681274,  992.20592869],
        [ 994.01125247,  988.22293807,  989.68967838,  993.393716  ],
        [ 990.66950931,  995.34289169,  999.16309764,  997.61933578]],

       [[1001.92128515,  992.21590895, 1001.03328445, 1002.37262482],
        [1002.1061563 ,  994.19933894,  997.34679402, 1002.51909023],
        [1002.14300631, 1003.47905083,  997.60738123, 1009.92212443]],

       [[ 992.28932444, 1003.51508065,  988.90427335, 1000.35929282],
        [1001.80190108,  996.03234018,  996.63591226, 1004.63443074],
        [ 993.13155643,  990.40057918,  991.26890895, 1005.32344238]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

... or using dictionary access like this:


In [16]:
ds["Pressure"]


<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[ 999.74093538,  998.31870626,  991.95144368, 1003.27487984],
        [ 999.34766328, 1000.99127862, 1004.41242884, 1001.31141224],
        [1003.71698511,  994.01681277, 1010.9312329 ,  998.11440997]],

       [[ 997.02633328,  999.53683948, 1002.09920067, 1003.90272932],
        [ 999.54427246,  994.26087343,  995.18843132, 1006.53230269],
        [1004.83661616,  995.95255326,  994.60954819,  995.94156946]],

       [[ 992.48242962,  997.10037573,  991.17681274,  992.20592869],
        [ 994.01125247,  988.22293807,  989.68967838,  993.393716  ],
        [ 990.66950931,  995.34289169,  999.16309764,  997.61933578]],

       [[1001.92128515,  992.21590895, 1001.03328445, 1002.37262482],
        [1002.1061563 ,  994.19933894,  997.34679402, 1002.51909023],
        [1002.14300631, 1003.47905083,  997.60738123, 1009.92212443]],

       [[ 992.28932444, 1003.51508065,  988.90427335, 1000.35929282],
        [1001.80190108,  996.03234018,  996.63591226, 1004.63443074],
        [ 993.13155643,  990.40057918,  991.26890895, 1005.32344238]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

We'll return to the `Dataset` object when we start loading data from files in later tutorials today.


# Summary

In this initial tutorial, the `DataArray` and `Dataset` objects were utilized to create and explore synthetic examples of climate data.


# Resources


Code and data for this tutorial is based on existing content from [Project Pythia](https://foundations.projectpythia.org/core/xarray/xarray-intro.html).
